In [17]:
import random
import json
import torch
import torch.nn as nn
import torchtext

ModuleNotFoundError: No module named 'torchtext'

In [7]:
class BiGRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        super(BiGRU, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, text):
        # text: [seq_len, batch_size]
        embedded = self.embedding(text) # [seq_len, batch_size, embedding_dim]
        embedded = self.dropout(embedded)
        hidden, _ = self.gru(embedded) # [seq_len, batch_size, hidden_dim*2]
        # Concatenate the forward and backward hidden states
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1) # [batch_size, hidden_dim*2]
        out = self.fc(hidden) # [batch_size, output_dim]
        return out

In [8]:
dataset_base_path = "../data/clevr/CLEVR_CoGenT_v1.0/questions/"
x_train = json.load(open(dataset_base_path + "CLEVR_trainA_questions.json", "r"))
x_val = json.load(open(dataset_base_path + "CLEVR_valA_questions.json", "r"))

In [12]:
len(x_train["questions"])

699960

In [16]:
x_train["questions"][random.randint(0, len(x_train["questions"]))]

{'question_index': 476847,
 'question_family_index': 10,
 'image_index': 47687,
 'question': 'Is the tiny shiny block the same color as the tiny matte ball?',
 'answer': 'no',
 'image_filename': 'CLEVR_trainA_047687.png',
 'split': 'trainA',
 'program': [{'value_inputs': [], 'inputs': [], 'function': 'scene'},
  {'value_inputs': ['small'], 'inputs': [0], 'function': 'filter_size'},
  {'value_inputs': ['metal'], 'inputs': [1], 'function': 'filter_material'},
  {'value_inputs': ['cube'], 'inputs': [2], 'function': 'filter_shape'},
  {'value_inputs': [], 'inputs': [3], 'function': 'unique'},
  {'value_inputs': [], 'inputs': [], 'function': 'scene'},
  {'value_inputs': ['small'], 'inputs': [5], 'function': 'filter_size'},
  {'value_inputs': ['rubber'], 'inputs': [6], 'function': 'filter_material'},
  {'value_inputs': ['sphere'], 'inputs': [7], 'function': 'filter_shape'},
  {'value_inputs': [], 'inputs': [8], 'function': 'unique'},
  {'value_inputs': [], 'inputs': [4], 'function': 'query_c

In [ ]:
vocab = []
for question in x_train["questions"]:
    